In [1]:
from datasets import load_dataset, load_metric, load_from_disk, concatenate_datasets
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer
from evaluate import load
import os
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict

In [ ]:
#stream_dataset_train = load_dataset('librispeech_asr', 'clean', split='train.100', streaming=True)
#stream_dataset_val = load_dataset('librispeech_asr', 'clean', split='validation', streaming=True)

#libri = DatasetDict({'train.100':Dataset.from_list(list(stream_dataset_train.take(5000))),
                     #'validation':Dataset.from_list(list(stream_dataset_val.take(500)))})
#print(libri)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Dataset

In [3]:
libri= load_dataset("librispeech_asr", name="clean")
print(libri)

Found cached dataset librispeech_asr (/home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/cff5df6e7955c80a67f80e27e7e655de71c689e2d2364bece785b972acb37fe7)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train.100: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 28539
    })
    train.360: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 104014
    })
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2703
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2620
    })
})


In [ ]:
#libri.save_to_disk('/home/ujan/Desktop/libri')
#libri = load_from_disk('/home/ujan/Desktop/libri')
#libri

In [4]:
libri = libri.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(libri["train.100"].remove_columns(["file", "audio"]))

,text
0,AS OF COURSE THE COLD PENETRATES MORE EASILY THROUGH THE THIN SKIN AND GIVES RISE TO DAMPNESS IN THE FORM OF RIME ON MEETING THE WARMTH OF THE BODY THESE THIN PATCHES REMAIN DAMP WHENEVER ONE IS IN THE BAG AND IN A SHORT TIME THEY LOSE THEIR HAIR
1,NOT SO WELL THE FLOOR OF THE CABIN THAT IS THE HOTEL WAS A GREAT DEAL HARDER THAN THE SAND SPIT WHERE I HAD PASSED THE NIGHT I HAD PLENTY OF PURE FRESH AIR WHILE SHE IN A CLOSED CABIN
2,ARE THE GOLDEN AGE AND DREAM DAYS AM I NOT RIGHT THANK YOU BUT THE BOOK YOU HAVE NOT READ MY BOOK
3,CHIRON THE CENTAUR HAD MET HIM AS HE WAS WANDERING THROUGH THE FORESTS ON THE MOUNTAIN PELION AND HAD SENT HIM DOWN INTO IOLCUS THEN THERE CAME TWO MEN WELL SKILLED IN THE HANDLING OF SHIPS TIPHYS AND NAUPLIUS
4,HE GAVE A TELEGRAPHIC CALL FOR HELP ON THE STEAM PIPE WHICH RUNS THROUGH THERE AND CONNECTS WITH THE WHISTLE THE LAD EXPLAINED I WAS ON DECK AND HEARD IT I TALKED WITH HIM OVER THE PIPE THERE IS NO TIME TO LOSE THEN COME WITH ME AND THE CAPTAIN HIMSELF HURRIEDLY LED THE WAY DOWN THROUGH THE LOWER DEPTHS OF THE SHIP
5,BUT AN INFERNAL AND TRULY STYGIAN SLEEP WHICH BEING THUS SET FREE FROM ITS PRISON TOOK POSSESSION OF HER AND SHE FELL DOWN IN THE MIDST OF THE ROAD A SLEEPY CORPSE WITHOUT SENSE OR MOTION BUT CUPID BEING NOW RECOVERED FROM HIS WOUND
6,AND RESOLVED TO SHOW THAT THE PIANO TOO COULD PRODUCE THRILLING EFFECTS IN THE SAME WAY HE HAD LISTENED TO THE HUMAN VOICE AND DETERMINED THAT THE SONG OF HIS OWN INSTRUMENT SHOULD BE HEARD
7,THEY REFLECT THE SPIRIT OF ASSYRIA'S GREATNESS WHICH HOWEVER HAD A MATERIALISTIC BASIS ASSYRIAN ART FOUND EXPRESSION IN DELINEATING THE OUTWARD FORM RATHER THAN IN STRIVING TO CREATE A THING OF BEAUTY WHICH IS A JOY FOR EVER
8,AND THAT THE VERY IDEAS OF SEVENTEEN EIGHTY NINE WERE ARRESTED IN THEIR DEVELOPMENT BY NAPOLEON'S CAPRICE THE IDEAS OF THE REVOLUTION AND THE GENERAL TEMPER OF THE AGE PRODUCED NAPOLEON'S POWER BUT NAPOLEON'S POWER SUPPRESSED THE IDEAS OF THE REVOLUTION
9,PRECISELY HERE WE ARE MEN OF DUTY EVEN WE OCCASIONALLY IT IS TRUE WE DANCE IN OUR CHAINS AND BETWIXT OUR SWORDS


In [6]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

libri = libri.map(remove_special_characters)

Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/cff5df6e7955c80a67f80e27e7e655de71c689e2d2364bece785b972acb37fe7/cache-1e923047bb94a24d.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/cff5df6e7955c80a67f80e27e7e655de71c689e2d2364bece785b972acb37fe7/cache-b52cf0b8414b877c.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/cff5df6e7955c80a67f80e27e7e655de71c689e2d2364bece785b972acb37fe7/cache-a5d0ed8624a22c61.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/cff5df6e7955c80a67f80e27e7e655de71c689e2d2364bece785b972acb37fe7/cache-5f0efa38cd301d02.arrow


In [7]:
show_random_elements(libri["train.100"].remove_columns(["file", "audio"]))

,text
0,the council of buccaneers had lasted some time when one of them re entered the house and with a repetition of the same salute which had in my eyes an ironical air begged for a moment's loan of the torch silver briefly agreed
1,retire therefore to your chamber and repose on your bed of down and when you see fit repair to the bath supper awaits you in the adjoining alcove when it pleases you to take your seat there psyche gave ear to the admonitions of her vocal attendants
2,i know she would not shoot one of those big guns for a dozen elk besides that she is very tender hearted and will never harm anything herself although she likes to join our hunts i think you must be tired of this letter so i am going to say good night my friend
3,unless i may rescue that noble lady then eagerly he asked his armor and put it on him and mounted his horse and rode as fast as he might
4,as for her own raiment the wives of few artisans would have been content to put up with missus crawley's best gown the stuff of which it was made had been paid for by her mother when she with much difficulty bestowed upon her daughter her modest wedding trousseau
5,as thus he strode along in anger putting together the words he would use to chide little john he heard of a sudden loud and angry voices
6,it had rallied widespread public support to the amend ment as a war measure and had itself become almost univer sally accepted if not universally approved and in the midst of picketing ands in spite of all the prophecies and fears that picketing would set back the cause
7,but d'artagnan did not stir
8,which stood at the door breathless with haste i was yet able to call out put that table down directly at the same moment mister brownrigg appeared from within the door
9,we take it all back jerry vowed after this we'll own up that you know a bull moose from a mule or a buck deer every time that's going to be a prize picture all right those last words from frank made will very proud


Extract all distinct letters of the training and test data and build our vocabulary from this set of letters

In [8]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = libri.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=libri.column_names["train.100"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
vocab_list = list(set(vocabs["train.100"]["vocab"][0]) | set(vocabs["validation"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'t': 0,
 'o': 1,
 'v': 2,
 'y': 3,
 'r': 4,
 'q': 5,
 'f': 6,
 'i': 7,
 'a': 8,
 'g': 9,
 "'": 10,
 'e': 11,
 'b': 12,
 ' ': 13,
 'k': 14,
 'p': 15,
 'j': 16,
 'c': 17,
 'x': 18,
 'd': 19,
 'u': 20,
 's': 21,
 'w': 22,
 'l': 23,
 'm': 24,
 'z': 25,
 'h': 26,
 'n': 27}

To make it clearer that " " has its own token class, we give it a more visible character |. In addition, we also add an "unknown" token. We also add a padding token that corresponds to CTC's "blank token"

In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


Save the vocabulary as a json file

In [14]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.

In [15]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Wav2Vec2 feature extractor

In [16]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

In [17]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [18]:
#libri['train.100'][0]

In [19]:
#rand_int = random.randint(0, len(libri["train.100"]))

#print(libri["train.100"][rand_int]["text"])
#ipd.Audio(data=np.asarray(libri["train.100"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

In [20]:
#rand_int = random.randint(0, len(libri["train.100"]))

#print("Target text:", libri["train.100"][rand_int]["text"])
#print("Input array shape:", np.asarray(libri["train.100"][rand_int]["audio"]["array"]).shape)
#print("Sampling rate:", libri["train.100"][rand_int]["audio"]["sampling_rate"])

In [21]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [22]:
libri = libri.map(prepare_dataset, remove_columns=libri.column_names["train.100"], num_proc=4)

#0:   0%|          | 0/7135 [00:00<?, ?ex/s]

#1:   0%|          | 0/7135 [00:00<?, ?ex/s]

#2:   0%|          | 0/7135 [00:00<?, ?ex/s]

#3:   0%|          | 0/7134 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labe

...

#0:   0%|          | 0/655 [00:00<?, ?ex/s]

#1:   0%|          | 0/655 [00:00<?, ?ex/s]

#2:   0%|          | 0/655 [00:00<?, ?ex/s]

#3:   0%|          | 0/655 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labe

Training & Evaluation

In [23]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [52]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [54]:
model = Wav2Vec2ForCTC.from_pretrained(
    "/home/ujan/Downloads/w2v2_85k",
    #"facebook/wav2vec2-base", 
    #"patrickvonplaten/wav2vec2-base-repro-960h-libri-85k-steps"
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of the model checkpoint at /home/ujan/Downloads/w2v2_85k were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'project_q.bias', 'project_hid.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/ujan/Downloads/w2v2_85k and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use i

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore 

In [55]:
model.freeze_feature_encoder()

In [56]:
train_dataloader = DataLoader(
    libri["train.100"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=32, # 32
)

val_dataloader = DataLoader(
    libri["validation"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=32, # 32
)

optimizer = AdamW(model.parameters(),
                  lr=1e-4,
                  weight_decay=0.005)

num_train_epochs = 30
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=1000,
    num_training_steps=num_training_steps,
)

In [57]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

In [ ]:
progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda")
model.to(device)

step = 1

for epoch in range(num_train_epochs):

    val_loss = 0
    # Training
    model.train()
    for batch in train_dataloader:
        inputs = batch['input_values'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_values=inputs, labels=labels)      
        loss = outputs.loss
        train_loss = loss.item()
        loss.backward()

        optimizer.step()
        #lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        if step % 500 == 0:  
            val_loss = 0
            for batch in val_dataloader:
                inputs = batch['input_values'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_values=inputs, labels=labels)
                logits = outputs.logits
                val_loss += outputs.loss.item()
                
                pred_ids = torch.argmax(logits, dim=-1)

                labels[labels == -100] = processor.tokenizer.pad_token_id

                pred_str = processor.batch_decode(pred_ids)
                # we do not want to group tokens when computing the metrics
                label_str = processor.batch_decode(labels, group_tokens=False)

                wer_metric.add_batch(predictions=pred_str, references=label_str)
                cer_metric.add_batch(predictions=pred_str, references=label_str)
                
            wer_result = wer_metric.compute()
            cer_result = cer_metric.compute()
            print('Step: {}, Val loss: {}, wer: {}, cer: {}'.format(step,
                                                                    val_loss/len(val_dataloader),
                                                                    wer_result,
                                                                    cer_result))

            #### save ####
                  
        step += 1
        
    lr_scheduler.step()

  0%|          | 0/26760 [00:00<?, ?it/s]

/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
Step: 26500, Val loss: 0.2749312400817871, wer: 0.2319767655600897, cer: 0.06502898188978562 # fb


Evaluation

In [ ]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
    return batch

results = libri["test"].map(map_to_result, remove_columns=libri["test"].column_names)

  0%|          | 0/2620 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.082


In [ ]:
show_random_elements(results)

,pred_str,text
0,my heart doth plead that thou in him dost lie a closet never pierced with crystal eyes but the defendant doth that plead deniye and says in him thy fair apparants lies,my heart doth plead that thou in him dost lie a closet never pierc'd with crystal eyes but the defendant doth that plea deny and says in him thy fair appearance lies
1,she was here just now said the count,she was here just now said the count
2,but hackey who was too much occupied with his own thoughts to note the movement continued speaking more to himself than to his companion,but hawkeye who was too much occupied with his own thoughts to note the movement continued speaking more to himself than to his companion
3,a word sha now be said about the origin of luther's commonto agelations,a word should now be said about the origin of luther's commentary on galatians
4,you have received us with all that courtesy and hospitality for which your character in england stands so high,you have received us with all that courtesy and hospitality for which your character in england stands so high
5,so we harried the cost of norway,so we harried the coast of norway
6,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such scientists and discoverers as engineer simon lake ochinographer willam bb poler traveler sar earnist shaleton,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such scientists and discoverers as engineer simon lake oceanographer william beebe polar traveler sir ernest shackleton
7,tabby had tended them in their childhood they and none others should tend her in her infirmity and age,tabby had tended them in their childhood they and none other should tend her in her infirmity and age
8,mistress fitzouth had been carried off bo the sheriff's daughter and her maids as soon as they had entered the house so that robin alone wat the care of mont fichet,mistress fitzooth had been carried off by the sheriff's daughter and her maids as soon as they had entered the house so that robin alone had the care of montfichet
9,said captain donnethorne seating himself where he could see along the short passage to the open derry door,said captain donnithorne seating himself where he could see along the short passage to the open dairy door


In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(libri["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] c [PAD] [PAD] [PAD] [PAD] [PAD] o o o [PAD] n n [PAD] [PAD] [PAD] c c [PAD] [PAD] [PAD] [PAD] o o o r r r r d d | | r r [PAD] e [PAD] [PAD] [PAD] t t [PAD] [PAD] [PAD] [PAD] [PAD] u u r n n e d | | t o o | | | i [PAD] t s s s | | | p p l l [PAD] [PAD] [PAD] a a [PAD] c c e e [PAD] | | [PAD] [PAD] a [PAD] [PAD] [PAD] m m i i i d d d s s t t | | | t t h e e | | t t [PAD] [PAD] [PAD] [PAD] [PAD] e e n n n n [PAD] t [PAD] s s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
torch.save(model.state_dict(), '/home/ujan/Notebooks/models/wav2vec2_asr_libri_100')

In [ ]:
progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda")
model.to(device)

step = 1

for epoch in range(num_train_epochs):

    val_loss = 0
    # Training
    model.train()
    for batch in train_dataloader:
        inputs = batch['input_values'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_values=inputs, labels=labels)      
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        if step % 5 == 0:

            # Evaluation
            model.eval()
            for batch in val_dataloader:
                with torch.no_grad():
                    inputs = batch['input_values'].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_values=inputs, labels=labels)
                    val_loss += outputs.loss.item()

                pred_logits = outputs.logits
                pred_ids = torch.argmax(pred_logits, dim=-1)
                label_ids = batch['labels']
                label_ids[labels == -100] = processor.tokenizer.pad_token_id

                pred_str = processor.batch_decode(pred_ids)
                # we do not want to group tokens when computing the metrics
                label_str = processor.batch_decode(label_ids, group_tokens=False)
                
                print(pred_str)
                print(label_str)
                raise

                wer.add_batch(predictions=pred_str, references=label_str)
                cer.add_batch(predictions=pred_str, references=label_str)

            wer_result = wer.compute()
            cer_result = cer.compute()
            print('step : {}, wer : {}, cer : {}'.format(step, wer_result, cer_result))
            print(val_loss/len(val_dataloader))
            
            #### save ####
                  
        step += 1

  0%|          | 0/53520 [00:00<?, ?it/s]

/home/ujan/anaconda3/envs/asr/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


['nwn n nwnwn nwn [UNK] nw[UNK]n[UNK]nbnb n[UNK]n[UNK] [UNK] [UNK]cbc[UNK]', 'n n cebn n nbnbnwcn n nw nen n unb nubnbn n b bcb[UNK]bc', 'n n n nwn w[UNK]bn[UNK]wnbn n cn nwnbwnenw[UNK]n n n w[UNK]wnwnwnwn n[UNK]n n n n n n n n wnwnw w[UNK]n u n nwn[UNK]n[UNK]w[UNK] n n cn[UNK]w n n[UNK]nb[UNK]n n nwn wnwnwnwbnwn bn n nc', "uncncnc n nwn nwnwncn n unw'n n[UNK]wc u[UNK]wnw wnwun nucn n nu ncb nbn[UNK] n c w en wnbwnwnwn n n [UNK]un n n ncnwn n ncncncn nc cn ncncnwn n n nc nun cnwncn nun ncwnwn wnbnwn n[UNK]nwnbnb cnbnbncn n n n n n ncwcn n n nbnwnb b cncnc c''c", "n n n cn bwncn[UNK]bn nwn[UNK]nbnebn wnwn[UNK]n nb[UNK] n[UNK]n[UNK]n[UNK]cnw'n nbn [UNK] [UNK]nbnb[UNK]n n[UNK]n n 'ncb[UNK]bc[UNK]", "n n n n' nbn n nwn n n nwn nwn [UNK]n n[UNK]n n[UNK]n n[UNK] [UNK]n n [UNK] n n c[UNK]c", "n n n n wnbnwn[UNK] n nwn[UNK]n n n n[UNK] wnwnwnebwbn[UNK]'nwnbnc c'cb[UNK]bc[UNK]", '[UNK]n nwnwnwn nbn n nwn nuwb[UNK]bwn ncbc', "n n n n n nwbn w' n n n n n n n nbnb n n cbc", "n n nbn n n[UNK]cnwn n

RuntimeError: No active exception to reraise